In [94]:
import numpy as np
import pandas as pd
import os

In [97]:
articles_metadata = pd.read_csv("../data/articles_metadata.csv")
articles_metadata.head(5)

,year,authors,journal,title,type,pdf?,keywords,keywords_online?,notes
0,2011,Bos et al.,Political Communication,How the Media Shape Perceptions of Right-Wing ...,article,y,y,y,NaN
1,2019,Blassnig et al.,Political Communication,Hitting a Nerve: Populist News Articles Lead t...,article,y,y,y,NaN
2,2017,Caramani,American Political Science Review,Will vs. Reason: The Populist and Technocratic...,article,y,y,NaN,NaN
3,2020,"Wuttke,Schimpf, Schoen",American Political Science Review,When the Whole Is Greater than the Sum of Its ...,article,y,y,NaN,NaN
4,2014,Treib,Journal of European Public Policy,"The voter says no, but nobody listens: causes ...",article,y,y,NaN,NaN


In [98]:
(articles_metadata["pdf?"]=="y").sum()

533

In [99]:
filepaths = []
for root, dir, files in os.walk("../data/large_data/articles"):
    journal = root.rpartition("/")[2]
    filepaths.extend([journal + "/" + fname for fname in files if ".pdf" in fname])
filepaths[:10]

['American Political Science Review/2017 - Caramani - Will vs. Reason_The Populist and Technocratic Forms of Political Representation and Their Critique to Party Government.pdf',
 'American Political Science Review/2020 - Wuttke, Schimpf, Schoen - When the Whole Is Greater than the Sum of Its Parts.pdf',
 'Political Research Quarterly/2020 - Castanho Silva et al. - An Empirical Comparison of Seven Populist Attitudes Scales.pdf',
 'Political Research Quarterly/2018 - Peel - The Populist Theory of the State in Early American Political Thought.pdf',
 'Political Research Quarterly/2020 - Kenny - “The Enemy of the People”_Populists and Press Freedom.pdf',
 'Political Research Quarterly/2018 - Fumurescu - The People’s Two Bodies_An Alternative Perspective on Populism and Elitism.pdf',
 'Political Research Quarterly/2016 - Spruyt et al. - Who Supports Populism and What Attracts People to It.pdf',
 'Political Research Quarterly/2018 - Love, Windsor - Populism and Popular Support_Vertical Accou

In [203]:
def get_file_path(row):
    # first start from title
    title_candidates = [fp for fp in filepaths if row["title"].lower()[:8] in unicodedata.normalize("NFC", fp.lower())]
    if len(title_candidates) > 1:
        title_candidates = [fp for fp in title_candidates if row["authors"].lower() in unicodedata.normalize("NFC", fp.lower())]
        #title_candidates = [fp for fp in title_candidates if row["authors"].split(",")[0].lower() in unicodedata.normalize("NFC", fp.lower())]
    if len(title_candidates) > 1:
        title_candidates = [fp for fp in title_candidates if str(row["year"]) in fp.lower()]
    if len(title_candidates) > 1:
        title_candidates = [fp for fp in title_candidates if (row["journal"].lower()[:15]) in unicodedata.normalize("NFC", fp.lower())]
    # start again from author
    if len(title_candidates) == 0:
        title_candidates = [fp for fp in filepaths if row["authors"].split(",")[0].lower() in unicodedata.normalize("NFC", fp.lower())]
        if len(title_candidates) > 1:
            title_candidates = [fp for fp in title_candidates if str(row["year"]) in fp.lower()]
        if len(title_candidates) > 1:
            title_candidates = [fp for fp in title_candidates if (row["journal"].lower()[:10]) in unicodedata.normalize("NFC", fp.lower())]
        if len(title_candidates) > 1:
            title_candidates = [fp for fp in title_candidates if row["title"].lower()[:8] in unicodedata.normalize("NFC", fp.lower())]
        if len(title_candidates) > 1:
            title_candidates = [fp for fp in title_candidates if row["title"].lower()[-8:] in unicodedata.normalize("NFC", fp.lower())]
    # if still unsuccessful, then start with journal title
    if len(title_candidates) == 0:
        title_candidates = [fp for fp in filepaths if row["journal"].lower() in unicodedata.normalize("NFC", fp.lower())]
        if len(title_candidates) > 1:
            title_candidates = [fp for fp in title_candidates if str(row["year"]) in fp]
        if len(title_candidates) > 1:
            title_candidates = [fp for fp in title_candidates if row["title"].lower()[:6] in unicodedata.normalize("NFC", fp.lower())]
        if len(title_candidates) > 1:
            title_candidates = [fp for fp in title_candidates if row["authors"].lower()[:8] in unicodedata.normalize("NFC", fp.lower())]
    return title_candidates
    #if bool(title_candidates):
    #    return title_candidates[0]
    #else:
    #    return np.nan

articles_metadata["filepath"] = articles_metadata.apply(lambda row: get_file_path(row), axis=1)

In [207]:
(articles_metadata["filepath"].apply(len) ==0).sum()

5

In [208]:
(articles_metadata["filepath"].apply(len) > 1).sum()

0

In [210]:
missing = articles_metadata[articles_metadata["filepath"].apply(len) ==0]
missing

,year,authors,journal,title,type,pdf?,keywords,keywords_online?,notes,filepath
97,2017,Loeppky,New Political Economy,‘Adaptive accumulation’ and US political economy,article,y,y,NaN,NaN,[]
98,2020,"Bluhm, Varga",New Political Economy,Conservative Developmental Statism in East Cen...,article,y,y,NaN,NaN,[]
99,2020,Toplišek,New Political Economy,The Political Economy of Populist Rule in Post...,article,y,y,NaN,NaN,[]
270,2017,McDonnell,Political Studies,Populist Leaders and Coterie Charisma,article,y,y,NaN,NaN,[]
530,2020,Dewi,Contemporary Southeast Asia,Motherhood Identity in the 2019 Indonesian Pre...,article,n,y,NaN,NEJDE STÁHNOUT - NEMÁM PŘÍSTUP,[]


In [212]:
articles_metadata.to_json("../data/article_metadata_with_filepaths.json")